In [54]:
import pandas as pd
import requests
import math


def query(query, metrics=None, summary=True, m2=False):
    default = {
        "qf": {
            "key": 0,
            "book": 0.25,
            "path": 0.5,
            "text": 1,
            "text_raw": 1.5,
            "title": 2,
            "title_raw": 3,
        },
        "pf": {
            "key": 0,
            "book": 0.25,
            "path": 0.5,
            "text": 1,
            "text_raw": 1.5,
            "title": 2,
            "title_raw": 3,
        },
        "consolidado": 4,
        "tie": 0,
    }
        
    # if m2 then one url, else the other
    if m2:
        url = "http://localhost:5003/solr/articles/select"
        metrics = default
    else:
        url = "http://localhost:5002/solr/articles/select"

    if metrics is None:
        metrics = default

    x = requests.get(
        url=url,
        params={
            "defType": "edismax",
            "qf": f"book^{metrics['qf']['book'] or default['qf']['book']} key^{metrics['qf']['key'] or default['qf']['key']} path^{metrics['qf']['path'] or default['qf']['path']} text^{metrics['qf']['text'] or default['qf']['text']} text_raw^{metrics['qf']['text_raw'] or default['qf']['text_raw']} title^{metrics['qf']['title'] or default['qf']['title']} title_raw^{metrics['qf']['title_raw'] or default['qf']['title_raw']}",
            "pf": f"book^{metrics['pf']['book'] or default['pf']['book']} key^{metrics['pf']['key'] or default['pf']['key']} path^{metrics['pf']['path'] or default['pf']['path']} text^{metrics['pf']['text'] or default['pf']['text']} text_raw^{metrics['pf']['text_raw'] or default['pf']['text_raw']} title^{metrics['pf']['title'] or default['pf']['title']} title_raw^{metrics['pf']['title_raw'] or default['pf']['title_raw']}",
            "bq": f"state:Consolidado^{metrics['consolidado'] or default['consolidado']}",
            "tie": f"{metrics['tie'] or default['tie']}",
            "q": query,
            "start": 0,
            "rows": 100
        }
    )

    res = x.json()['response']['docs']
    if summary:
        res = _parse_articles(res)

    return res


def _parse_articles(articles):
    # gets the last element of a list
    return [build_article(article['book'], article['key'], article['path'][-1], article['date']) for article in articles]


def build_article(book, key, last_path, date):
    return "{}/{}/{}/{}".format(book, key, last_path, date)

def _hits(results, relevant, k):
    # makes a copy of the articles
    relevant = relevant.copy()
    res = ""
    i = 0
    for result in results:
        if result in relevant:
            res += "1"
            # drops the result from the articles
            relevant.remove(result)
            # if articles is empty breaks
            if len(relevant) == 0:
                break
        else:
            res += "0"

        i += 1
        if i >= k:
            break
    return res

def _average_precision(results, relevant):
    precision_values = [
        len([
            doc
            for doc in results[:idx]
            if doc in relevant
        ]) / idx
        for idx in range(1, len(results))
    ]
    if len(precision_values) == 0:
        return 0
    return sum(precision_values)/len(precision_values)

def _precision_at_k(results, relevant, k):
    return len([doc for doc in results[:k] if doc in relevant])/k

def _recall(results, relevant):
    return len([doc for doc in results if doc in relevant])/len(relevant)


def _ndcg(results, relevant, order=False):
    """
    Compute the normalized discounted cumulative gain (NDCG) of the results
    against the expected results.

    The relevance grade is the inverse order of the list. E.g. [3, 2, 1]
    """
    len_expected = len(relevant)

    dcg = 0.0
    for i, result in enumerate(results):
        if result in relevant:
            if order:
                relevance_grade = len_expected - relevant.index(result)
            else:
                relevance_grade = 1
            dcg += (2 ** relevance_grade - 1) / math.log(i + 2, 2)
    idcg = 0.0
    for i, result in enumerate(relevant):
        if order:
            relevance_grade = len_expected - i
        else:
            relevance_grade = 1
        idcg += (2 ** relevance_grade - 1) / math.log(i + 2, 2)

    return dcg / idcg

def _build_tables(queries, metrics, m2):
    df = pd.DataFrame(
        columns=['hits @ 10', 'average precision', 'precision @ 10', 'recall', 'ndcg'])

    # for each query, run the metrics and add to the dataframe
    for str, articles in queries.items():
        results = query(str, metrics=metrics, m2=m2)
        df.loc[str[:12]] = [
            _hits(results, articles, 10),
            _average_precision(results, articles),
            _precision_at_k(results, articles, 10),
            _recall(results, articles),
            _ndcg(results, articles)
        ]

    return df

def evaluate(metrics):
    queries = {
        "horas suplementares": [
            build_article("Código do Trabalho", 226,
                          "Subsecção VII Trabalho suplementar", "2012-08-01T00:00:00Z"),
            build_article("Código do Trabalho", 227,
                          "Subsecção VII Trabalho suplementar", "2009-02-12T00:00:00Z"),
            build_article("Código do Trabalho", 228,
                          "Subsecção VII Trabalho suplementar", "2009-02-12T00:00:00Z"),
            build_article("Código do Trabalho", 229,
                          "Subsecção VII Trabalho suplementar", "2012-08-01T00:00:00Z"),
            build_article("Código do Trabalho", 230,
                          "Subsecção VII Trabalho suplementar", "2012-08-01T00:00:00Z"),
            build_article("Código do Trabalho", 231,
                          "Subsecção VII Trabalho suplementar", "2009-02-12T00:00:00Z")
        ],
        "+socialismo +date:[1976-01-01T00:00:00Z TO 1976-12-31T23:59:59Z] +book:constituicao": [
            build_article("Constituição da República Portuguesa", 185,
                          "Capítulo I Função e estrutura", "1976-04-10T00:00:00Z"),
            build_article("Constituição da República Portuguesa",
                          2, "Princípios fundamentais", "1976-04-10T00:00:00Z"),
            build_article("Constituição da República Portuguesa", 273,
                          "Título X Defesa Nacional", "1976-04-10T00:00:00Z"),
            build_article("Constituição da República Portuguesa", 89,
                          "Título I Princípios gerais", "1976-04-10T00:00:00Z"),
        ],
        "horario flexivel": [
            build_article("Código do Trabalho", 56,
                          "Subsecção IV Parentalidade", "2015-09-06T00:00:00Z"),
            build_article("Código do Trabalho", 56,
                          "Subsecção IV Parentalidade", "2009-02-12T00:00:00Z"),
            build_article("Código do Trabalho", 57,
                          "Subsecção IV Parentalidade", "2009-02-12T00:00:00Z"),
        ],
        "artigo 1 da constituicao": [
            build_article("Constituição da República Portuguesa",
                          1, "Princípios fundamentais", "1989-08-07T00:00:00Z"),
            build_article("Constituição da República Portuguesa",
                          1, "Princípios fundamentais", "1976-04-10T00:00:00Z"),
        ],
        "art 2 registo civil": [
            build_article("Código do Registo Civil", 2,
                          "Capítulo I Objecto e valor do registo civil", "1995-06-06T00:00:00Z"),
            build_article("Código do Registo Civil", 2,
                          "Diploma", "1995-06-06T00:00:00Z"),
        ],
        "ideologia fascista": [
            build_article("Constituição da República Portuguesa", 163,
                          'Capítulo II Competência', '1976-04-10T00:00:00Z'),
            build_article("Constituição da República Portuguesa", 160,
                          'Capítulo I Estatuto e eleição', '1997-10-05T00:00:00Z'),
            build_article("Constituição da República Portuguesa", 46,
                          'Capítulo I Direitos, liberdades e garantias pessoais', '1982-10-30T00:00:00Z'),
            build_article("Constituição da República Portuguesa", 46,
                          'Capítulo I Direitos, liberdades e garantias pessoais', '1976-04-10T00:00:00Z'),
            build_article("Constituição da República Portuguesa", 46,
                          'Capítulo I Direitos, liberdades e garantias pessoais', '1997-10-05T00:00:00Z'),
        ],
        'art 10 Código Penal': [
            build_article('Código Penal', 10, 'Diploma',
                          '1995-03-15T00:00:00Z'),
            build_article(
                'Código Penal', 10, 'Capítulo I Pressupostos da punição', '1995-03-15T00:00:00Z'),
            build_article(
                'Código Penal', 10, 'Capítulo I Pressupostos da punição', '1998-09-07T00:00:00Z'),
        ],
        'interrupção voluntária da gravidez': [
            build_article('Código do Registo Civil',
                          '209 A', 'Subsecção V Morte fetal', '2002-04-25T00:00:00Z'),
            build_article('Código Penal', 142,
                          'Capítulo II Dos crimes contra a vida intra-uterina', '2007-04-22T00:00:00Z'),
            build_article('Código do Trabalho', 38,
                          'Subsecção IV Parentalidade', '2009-02-12T00:00:00Z'),
            build_article('Código Penal', 142,
                          'Capítulo II Dos crimes contra a vida intra-uterina', '1997-07-08T00:00:00Z'),
            build_article('Código Penal', 142,
                          'Capítulo II Dos crimes contra a vida intra-uterina', '1995-03-15T00:00:00Z'),
        ]
    }

    print("m2 👇")
    print(_build_tables(queries, metrics, True))

    print()
    print("m3 👇")
    print(_build_tables(queries, metrics, False))


In [55]:
metrics = {'qf': {'book': 38.92432103822159, 'key': 61.56994575348722, 'path': 7.058582346255768, 'text': 2.4731813053938483, 'text_raw': 7.229762219930129, 'title': 9.425539065132167, 'title_raw': 0.39651608022263407}, 'pf': {
    'book': 22.488846194363205, 'key': 0.9374478542140957, 'path': 22.387246976224922, 'text': 3.0407590739988253, 'text_raw': 17.2687750652928, 'title': 10.148322926042312, 'title_raw': 25.311744954858113}, 'consolidado': 30.0, 'tie': 0.2897121725263928}
evaluate(metrics)


m2 👇
               hits @ 10  average precision  precision @ 10  recall      ndcg
horas suplem  1111100000           0.213130             0.5     1.0  0.961104
+socialismo         1111           0.337108             0.4     1.0  1.000000
horario flex         111           0.153500             0.3     1.0  1.000000
artigo 1 da   0000000000           0.000000             0.0     0.0  0.000000
art 2 regist  0000000000           0.000000             0.0     0.0  0.000000
ideologia fa       11111           0.321587             0.5     1.0  1.000000
art 10 Códig  0000000000           0.000000             0.0     0.0  0.000000
interrupção        11111           0.196669             0.5     1.0  1.000000

m3 👇
               hits @ 10  average precision  precision @ 10  recall      ndcg
horas suplem  1111100000           0.218462             0.5     1.0  0.973986
+socialismo         1111           0.337108             0.4     1.0  1.000000
horario flex         111           0.153500          